# Optional Challenge: Ames Housing Price Prediction

This is an **optional** task for students who want an extra challenge.

You will build a model to predict **SalePrice** for houses in Ames, Iowa using **79 explanatory variables**.

**Skills to practice**
- Leakage-safe preprocessing with `Pipeline` + `ColumnTransformer`
- Creative feature engineering (domain-inspired features)
- Advanced regression techniques (start with Linear Regression or Decision Tree baseline, then Random Forest, then tune)
- Cross-validation and proper evaluation (RMSLE / RMSE on log-scale)

## Dataset
Files (already in this repo):
- `datasets/housing/optional/train.csv` (has `SalePrice`)
- `datasets/housing/optional/test.csv` (no `SalePrice`)
- `datasets/housing/optional/data_description.txt` (column definitions)

**Goal**: Train on `train.csv`, then evaluate your model using cross-validation. You can also predict on `test.csv` to see how your model performs on unseen data.

In [1]:
# Setup
from __future__ import annotations

from pathlib import Path

import numpy as np
import pandas as pd

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, RobustScaler

from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

In [2]:
# Load data
DATA_DIR = Path('datasets/housing/optional')
train_path = DATA_DIR / 'train.csv'
test_path = DATA_DIR / 'test.csv'

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

print('train shape:', train_df.shape)
print('test shape :', test_df.shape)
train_df.head()

train shape: (1460, 81)
test shape : (1459, 80)


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [3]:
# Basic checks
assert 'SalePrice' in train_df.columns, 'Expected target column SalePrice in train.csv'
assert 'SalePrice' not in test_df.columns, 'test.csv should not include SalePrice'

# Peek at target distribution
print(f'SalePrice statistics:')
print(train_df['SalePrice'].describe())
print(f'\nSkewness: {train_df["SalePrice"].skew():.3f}')

# Optional: peek at missingness
missing = (train_df.isna().mean().sort_values(ascending=False) * 100).head(15)
print('\nTop 15 columns with missing values:')
print(missing)

SalePrice statistics:
count      1460.000000
mean     180921.195890
std       79442.502883
min       34900.000000
25%      129975.000000
50%      163000.000000
75%      214000.000000
max      755000.000000
Name: SalePrice, dtype: float64

Skewness: 1.883

Top 15 columns with missing values:
PoolQC          99.520548
MiscFeature     96.301370
Alley           93.767123
Fence           80.753425
MasVnrType      59.726027
FireplaceQu     47.260274
LotFrontage     17.739726
GarageQual       5.547945
GarageFinish     5.547945
GarageType       5.547945
GarageYrBlt      5.547945
GarageCond       5.547945
BsmtFinType2     2.602740
BsmtExposure     2.602740
BsmtCond         2.534247
dtype: float64


## Evaluation
A common choice for this dataset is **RMSLE** (root mean squared log error).

In practice, you can optimize RMSE on the transformed target: $\log(1 + `SalePrice`)$.
That is:
- Train the model on `y_log = log1p(SalePrice)`
- Evaluate RMSE on `y_log` using cross-validation
- For final predictions, do `expm1(pred_log)` to get back to dollars

## Data Cleaning Tips (Optional)

Before moving to feature engineering, consider:

1. **Drop high-missing columns** (e.g., >70% missing): `PoolQC`, `MiscFeature`, `Alley`, `Fence`, `FireplaceQu`
   - Or understand what 'NA' means (often = 'No pool', 'No fence', etc.) and create binary indicators

2. **Handle outliers**: Some very large/expensive houses can skew models
   - Consider removing extreme outliers (e.g., GrLivArea > 4000 with low price)

3. **Skewness correction**: Many numeric features are right-skewed
   - Apply `log1p` or Box-Cox transformation to highly skewed features (skew > 0.75)

4. **Feature types**: Check if some numeric columns should be categorical
   - `MSSubClass`, `OverallQual`, `OverallCond` might work better as categories

In [ ]:
# Split features/target
X = train_df.drop(columns=['SalePrice'])
y = train_df['SalePrice']

# Optional (recommended): train on log1p target to align with RMSLE
y_log = np.log1p(y)

X_train, X_valid, y_train_log, y_valid_log = train_test_split(
    X, y_log, test_size=0.2, random_state=RANDOM_STATE
)

print('X_train:', X_train.shape, 'X_valid:', X_valid.shape)

## TODO 1: Build a leakage-safe preprocessing pipeline
Use `ColumnTransformer` to preprocess numeric and categorical features.

Suggested starting point:
- Numeric: `SimpleImputer(strategy='median')`
- Categorical: `SimpleImputer(strategy='most_frequent')` + `OneHotEncoder(handle_unknown='ignore')`

Tip: Let pandas dtypes select columns automatically.

In [ ]:
# TODO 1: Build a leakage-safe preprocessing pipeline

# Goal: create a ColumnTransformer named `preprocess` that:
# - imputes numeric features (median) and scales them with RobustScaler (robust to outliers)
# - imputes categorical features (most_frequent or constant='Missing') then one-hot encodes
# - uses `handle_unknown='ignore'` in OneHotEncoder

# Advanced tips:
# - RobustScaler is better than StandardScaler for data with outliers
# - For high-missing columns, consider dropping them first or using constant imputation
# - Apply skewness correction (log1p) to highly skewed numeric features before scaling

# Hints:
# - Start by identifying columns via pandas dtypes
#   numeric_features = X_train.select_dtypes(exclude=['object']).columns
#   categorical_features = X_train.select_dtypes(include=['object']).columns
# - Use Pipeline + ColumnTransformer

numeric_features = None  # TODO
categorical_features = None  # TODO

# Example structure (fill in the details):
numeric_transformer = None  # TODO: Pipeline([('imputer', SimpleImputer(strategy='median')),
#                                           ('scaler', RobustScaler())])

categorical_transformer = None  # TODO: Pipeline([('imputer', SimpleImputer(strategy='most_frequent')),
#                                                 ('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocess = None  # TODO: ColumnTransformer([('num', numeric_transformer, numeric_features),
#                                                ('cat', categorical_transformer, categorical_features)])

print("TODO: implement `preprocess` (ColumnTransformer) before moving on.")

## TODO 2: Baseline model (Linear Regression or Decision Tree)
Build a baseline to establish a CV score before doing anything fancy.

- Use a `Pipeline(preprocess -> LinearRegression)` or `Pipeline(preprocess -> DecisionTreeRegressor)`
- Evaluate with 5-fold CV using RMSE on `y_log`

In [ ]:
# TODO 2: Baseline model (Linear Regression or Decision Tree)

# Goal: build a Pipeline(preprocess -> LinearRegression or DecisionTreeRegressor) and evaluate CV RMSE on y_log.

# Hints:
# Option 1 - Linear Regression:
# - model = Pipeline(steps=[('preprocess', preprocess), ('model', LinearRegression())])
#
# Option 2 - Decision Tree:
# - model = Pipeline(steps=[('preprocess', preprocess), ('model', DecisionTreeRegressor(random_state=RANDOM_STATE))])
#
# Then use: cross_val_score(model, X, y_log, cv=5, scoring='neg_root_mean_squared_error')
# Convert to positive RMSE via the leading minus sign: rmse = -scores.mean()

baseline_model = None  # TODO

if preprocess is None:
    print("TODO: implement `preprocess` first.")
else:
    print("TODO: implement `baseline_model` and cross-validation.")

## TODO 3: Creative Feature Engineering (your turn)
Create **at least 3** new features that you believe improve predictive power.

Ideas (pick a few):
- Total square footage (basement + 1st + 2nd floor)
- House age: `YrSold - YearBuilt`
- Years since remodel: `YrSold - YearRemodAdd`
- Total bathrooms: `FullBath + 0.5*HalfBath + BsmtFullBath + 0.5*BsmtHalfBath`
- Indicator features: `HasGarage`, `HasBasement`, `HasFireplace`, `HasPool`

**Important**: Feature engineering should not use `SalePrice` (avoid leakage).

In [ ]:
# Skeleton: write a function that adds engineered features
def add_engineered_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Add domain-specific engineered features.
    
    Best practices for housing price prediction:
    1. Total area features (combine related areas)
    2. Age/time features (house age, remodel age)
    3. Quality aggregates (combine quality ratings)
    4. Interaction features (area × quality)
    5. Binary indicators (has feature or not)
    """
    df = df.copy()

    # TODO: implement at least 5 engineered features below.
    #
    # Suggested features (high-impact):
    #
    # 1. TOTAL AREAS:
    #    - TotalSF = TotalBsmtSF + 1stFlrSF + 2ndFlrSF
    #    - TotalPorchSF = OpenPorchSF + EnclosedPorch + 3SsnPorch + ScreenPorch
    #
    # 2. AGE FEATURES:
    #    - HouseAge = YrSold - YearBuilt
    #    - YearsSinceRemodel = YrSold - YearRemodAdd
    #    - GarageAge = YrSold - GarageYrBlt (watch for NaN)
    #
    # 3. QUALITY AGGREGATES:
    #    - TotalBath = FullBath + 0.5*HalfBath + BsmtFullBath + 0.5*BsmtHalfBath
    #    - TotalQual = OverallQual + OverallCond
    #
    # 4. BINARY INDICATORS:
    #    - HasGarage = (GarageArea > 0).astype(int)
    #    - HasBasement = (TotalBsmtSF > 0).astype(int)
    #    - HasFireplace = (Fireplaces > 0).astype(int)
    #    - Has2ndFloor = (2ndFlrSF > 0).astype(int)
    #    - HasPool = (PoolArea > 0).astype(int)
    #
    # 5. INTERACTION FEATURES (area × quality):
    #    - QualityTimesArea = OverallQual × GrLivArea
    #
    # 6. POLYNOMIAL FEATURES (for key predictors):
    #    - GrLivArea_Squared = GrLivArea²
    
    return df

# NOTE: Once implemented, sanity-check shape changes:
# X_fe = add_engineered_features(X_train)
# print('Before:', X_train.shape, 'After:', X_fe.shape)

## TODO 4: Advanced regression model — Random Forest
Train a Random Forest model (with preprocessing!) and evaluate with CV.

Tips:
- Random Forest doesn’t need feature scaling, but it benefits from good missing-value handling and encoded categoricals.
- Start simple, then tune: `n_estimators`, `max_depth`, `min_samples_leaf`, `max_features`.

In [ ]:
# TODO 4: Advanced regression model — Random Forest

# Goal: build a Pipeline(preprocess -> RandomForestRegressor) and evaluate CV RMSE on y_log.

# Hints:
# - rf = RandomForestRegressor(random_state=RANDOM_STATE, n_estimators=..., max_depth=..., n_jobs=-1)
# - rf_model = Pipeline(steps=[('preprocess', preprocess), ('model', rf)])
# - Use: cross_val_score(rf_model, X, y_log, cv=5, scoring='neg_root_mean_squared_error')

rf_model = None  # TODO

if preprocess is None:
    print("TODO: implement `preprocess` first.")
else:
    print("TODO: implement `rf_model` and cross-validation.")

## TODO 5: Hyperparameter tuning (RandomizedSearchCV)
Use `RandomizedSearchCV` to tune your Random Forest (or Decision Tree).

Deliverable:
- Print best CV score
- Print best params
- Keep a reference to the best estimator as `best_model`

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

# TODO 5: Hyperparameter tuning (RandomizedSearchCV)

# Goal: run RandomizedSearchCV on your model to find better hyperparameters.

# Hints:
# - Use param names like 'model__n_estimators' because the regressor is inside Pipeline step 'model'
# - Keep n_iter modest to save time (e.g., 10–30)
# - Can tune DecisionTree (model__max_depth, model__min_samples_split) or 
#   RandomForest (model__n_estimators, model__max_depth, etc.)

# Example parameter grid for Random Forest:
param_distributions = {
    # TODO: fill in appropriate values
    # 'model__n_estimators': [100, 200, 500],
    # 'model__max_depth': [None, 10, 20, 30],
    # 'model__min_samples_leaf': [1, 2, 4],
    # 'model__max_features': ['sqrt', 'log2', None],
}

search = None  # TODO: RandomizedSearchCV(estimator=..., param_distributions=..., n_iter=20, cv=5, ...)

# TODO: run the search when ready
# search.fit(X, y_log)
# print('Best CV RMSE (log target):', -search.best_score_)
# print('Best params:', search.best_params_)
# best_model = search.best_estimator_

best_model = None

print("TODO: complete RandomizedSearchCV and set `best_model`.")

## TODO 6: Final Model Evaluation
Once you have a `best_model`, evaluate it on the validation set to get final performance metrics.

Optional: You can also predict on the test set to see how your model generalizes to completely unseen data (though you won't have true labels to compare against).

In [ ]:
# TODO 6: Final model evaluation

# Once you have a `best_model`, evaluate it properly

if best_model is None:
    print('TODO: set best_model (e.g., from RandomizedSearchCV) first.')
else:
    # Option 1: Evaluate on validation set
    best_model.fit(X_train, y_train_log)
    y_pred_log = best_model.predict(X_valid)
    
    # Calculate RMSE on log scale
    rmse_log = np.sqrt(mean_squared_error(y_valid_log, y_pred_log))
    print(f'Validation RMSE (log scale): {rmse_log:.4f}')
    
    # Convert back to original scale for interpretation
    y_valid_original = np.expm1(y_valid_log)
    y_pred_original = np.expm1(y_pred_log)
    rmse_original = np.sqrt(mean_squared_error(y_valid_original, y_pred_original))
    print(f'Validation RMSE (original scale): ${rmse_original:,.2f}')
    
    # Option 2: Train on full data and evaluate with CV
    # final_cv_score = cross_val_score(best_model, X, y_log, cv=5, 
    #                                   scoring='neg_root_mean_squared_error').mean()
    # print(f'Final CV RMSE: {-final_cv_score:.4f}')

## Optional extensions (extra challenge)

If you finish early and want to push further (still using only the techniques from this workshop), try:

### 1. Skewness Correction
- Identify highly skewed numeric features (skew > 0.75)
- Apply `log1p` transformation before preprocessing

### 2. Feature Selection (simple + practical)
- Drop high-missing columns (>70% missing)
- Use Random Forest feature importances to decide what to keep/drop

### 3. Ensembling (simple averaging)
- Train multiple models (Linear Regression, Decision Tree, RandomForest)
- Average predictions from multiple models

### 4. Outlier Handling
- Remove extreme outliers (e.g., very large living area with unusually low price)
- Compare CV score before/after removing outliers

### 5. Cross-Validation Strategy
- Use KFold with more folds (e.g., 10-fold) for more stable estimates